<h1>ASSIGNMENT 3 <br>
    COMPUTER VISION</h1>
<table align="left">
<tbody>
<tr>
<td style="width: 60%;">
<p>Presented by: Nathan Saric </p>
<p>Presented to: AccuWaste </p>
<p>Dataset used: <a href="https://www.kaggle.com/datasets/asdasdasasdas/garbage-classification"> Garbage Classification Dataset</a></p>
<p>Algorithm used: Image Classification </p>
</td>
<td style="width: 40%;">
<p>Weights</p>
<p>BUSINESS - 40%</p>
<p>TECHNICAL - 40%</p>
<p>PROFESSIONALISM - 20%</p>
</td>
</tr>
</tbody>
</table>

<h2>Company Description</h2>
<p style="text-align: justify;"> AccuWaste is a waste management company founded in the early 2000s that follows a business-to-business model. The company aims to provide waste-related services for residential, commercial, industrial, and municipal customers across North America including the collection, transfer, recycling, and disposal of waste. The company's vision is to become a leader in sustainability by minimizing environmental impact and discovering innovative environmental solutions. AccuWaste commits itself to key organizational values including accessibility, reliability, safety, and sustainability. The company's primary value driver lies in its industry expertise and broad network of waste management infrastructure that allows for efficient operations through sustainable technologies. Additionally, AccuWaste seeks to utilize its service offering to lead by example through its responsible stewardship of the environment and its social impact across the diverse customer base that AccuWaste serves. </p>

<h2>Problem Definition</h2>
<p style="text-align: justify;"> Having over 20 years of experience in the waste industry, AccuWaste has become a valuable business partner across major cities in North America. The company has made an ongoing effort to integrate its infrastructure within new cities and continues to expand its network and impact each year. Recently, the management team at AccuWaste has reached out to its research and development department to explore ways in which the company can further reduce its costs in order to support the company's growing operations while complying with the environmental regulations. The management team discovered that one of the major expenses that the company faced was its cost of disposal. A closer look revealed that this cost varies by material, with trash incineration being the more expensive way to manage waste. AccuWaste has identified six key classes of waste that its disposal facility regularly encounters: cardboard, glass, metal, paper, plastic, and trash. The company wishes to reduce the amount of recyclable material being unnecessarily incinerated or sent to landfills. Not only is this economically inefficient for the company, but this operational inefficiency hinders the company's potential in achieving its ambitious environmental goals, both short- and long-term. AccuWaste has further explained that cameras and sensors have recently been installed in all the waste sorting facilities. The company aims to implement these technologies to classify each item of waste that enters the facility and, thus, has reached out for a solution to maximize the accuracy of its waste classification system. </p>

<p style="text-align: justify;"> In this notebook, we explore the data relating to six classes of waste in an attempt to provide insights and actionable strategies on how the company can more accurately classify a given item in its sorting facility. We begin by processing and transforming the data. Given that the classification system equipped at the sorting facilities is new, AccuWaste has provided us with roughly 2,500 images to work with. Subsequently, we build and train a machine learning image classification algorithm to predict the class of waste for a particular image. Finally, we demonstrate how this model should be taken advantage of and further offer suggestions that aim to maximize the accuracy of the classification system, with an emphasis on risk and AccuWaste's key value drivers. </p>

<h2>Setup the Environment</h2>
<p style="text-align: justify;"> Before we can transform and load any data, we first initialize and configure a new <em>Amazon Sagemaker</em> <em>Session</em>. Additionally, we download several packages and tools in order to easily manipulate the datasets, such that they are compatible with <em>Amazon Sagemaker's Image Classification</em> algorithm. We will use <em>pip</em>, a package installer for Python, to install <em>unzip</em> and <em>mxnet</em>. The former is a tool that decompresses files and folders, whereas the latter is the package needed to create <em>.lst</em> files. Note that these augmented manifest files are the recommended input format for this particular algorithm. Furthermore, these packages only need to be installed once, otherwise, the console will notify that these requirements are already satisfied. Finally, we generate a new <em>Amazon S3 Bucket </em>to store the necessary input and output files later in the model construction stage. </p>

In [ ]:
import boto3
import sagemaker
from sagemaker.image_uris import retrieve 

# Creating an Amazon Sagemaker Session and creating variables for the Amazon Sagemaker S3 Bucket and Training Image
sess = sagemaker.Session() 
bucket = sess.default_bucket()
region = boto3.Session().region_name
prefix = 'garbage-classification-images'
training_image = retrieve(region=region, framework="image-classification")

In [ ]:
import os
import urllib.request

# Downloading the required tool for creating .lst files
def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)
        
download("https://raw.githubusercontent.com/apache/incubator-mxnet/master/tools/im2rec.py")

In [ ]:
# Installing the unzip utility tool needed to decompress files and folders
!pip install unzip

# Installing the mxnet package needed to create the .lst files
!pip install mxnet

<h2>Prepare the Data</h2>
<p style="text-align: justify;"> With our <em>Amazon</em> environment properly setup, we can now prepare the dataset. Note that we will be transforming the data locally on our computer, rather than in <em>Amazon Sagemaker, </em>to simplify the process. The "<em>Garbage Classification</em>" dataset contains 2,527 images across six folders that represent the six classes of waste under observation, namely, <em>Cardboard</em>, <em>Glass</em>, <em>Metal</em>, <em>Paper</em>, <em>Plastic</em>, and <em>Trash</em>. Each folder contains various images of the specific material category. We now partition the dataset by creating two new folders in the root folder, <em>garbage_train </em>and <em>garbage_test</em>, to designate the training and testing datasets, respectively. We will partition the data into an 80% training set and a 20% testing set, ensuring that each subfolder has the corresponding six class folders. The table below illustrates the resulting partitions for each class of data. For example, the 501 images of <em>Glass </em>were split, such that 401 images will be supplied to train the model, while 100 images will be used to test the model. Finally, we compress (or zip) the root folder, <em>garbage_classification</em>, to obtain the resulting <em>.zip</em> file that will be imported to our previously created environment. </p>


<table border="1" cellspacing="0" cellpadding="0">
<tbody>
<tr>
<td valign="top" width="91">
<p align="center"><strong>Class</strong></p></td>
<td valign="top" width="129">
<p align="center"><strong>Training (80%)</strong></p>
</td>
<td valign="top" width="121">
<p align="center"><strong>Testing (20%)</strong></p>
</td>
<td valign="top" width="112">
<p align="center"><strong>Total (100%)</strong></p>
</td>
</tr>
<tr>
<td valign="top" width="91">
<p align="right">Cardboard</p>
</td>
<td valign="top" width="129">
<p align="center">322</p>
</td>
<td valign="top" width="121">
<p align="center">81</p>
</td>
<td valign="top" width="112">
<p align="center">403</p>
</td>
</tr>
<tr>
<td valign="top" width="91">
<p align="right">Glass</p>
</td>
<td valign="top" width="129">
<p align="center">401</p>
</td>
<td valign="top" width="121">
<p align="center">100</p>
</td>
<td valign="top" width="112">
<p align="center">501</p>
</td>
</tr>
<tr>
<td valign="top" width="91">
<p align="right">Metal</p>
</td>
<td valign="top" width="129">
<p align="center">328</p>
</td>
<td valign="top" width="121">
<p align="center">82</p>
</td>
<td valign="top" width="112">
<p align="center">410</p>
</td>
</tr>
<tr>
<td valign="top" width="91">
<p align="right">Paper</p>
</td>
<td valign="top" width="129">
<p align="center">475</p>
</td>
<td valign="top" width="121">
<p align="center">119</p>
</td>
<td valign="top" width="112">
<p align="center">594</p>
</td>
</tr>
<tr>
<td valign="top" width="91">
<p align="right">Plastic</p>
</td>
<td valign="top" width="129">
<p align="center">385</p>
</td>
<td valign="top" width="121">
<p align="center">97</p>
</td>
<td valign="top" width="112">
<p align="center">482</p>
</td>
</tr>
<tr>
<td valign="top" width="91">
<p align="right">Trash</p>
</td>
<td valign="top" width="129">
<p align="center">109</p>
</td>
<td valign="top" width="121">
<p align="center">28</p>
</td>
<td valign="top" width="112">
<p align="center">137</p>
</td>
</tr>
<tr>
<td valign="top" width="91">
<p align="right"><strong>Total</strong></p>
</td>
<td valign="top" width="129">
<p align="center"><strong>2,020</strong></p>
</td>
<td valign="top" width="121">
<p align="center"><strong>507</strong></p>
</td>
<td valign="top" width="112">
<p align="center"><strong>2,527</strong></p>
</td>
</tr>
</tbody>
</table>

<p style="text-align: justify;"> After importing the compressed file containing the partitioned data from our local files, we will unzip the folder from within the environment using the <em>unzip</em> utility tool. Note that we choose to compress and decompress the dataset to avoid locally transferring thousands of individual image files to the environment which is extremely time-consuming and inefficient. Next, we run the <em>im2rec.py</em> Python script from <em>Apache mxnet </em>framework to convert the images into recordIO data format, a set of binary data exchange formats. This Python script will recursively iterate through all folders and subsequently output two files: <em>garbage_train.lst </em>and <em>garbage_test.lst</em>. These files are the manifest files that will be supplied to the algorithm in addition to all the images. Finally, we declare five variables that represent the location paths to the <em>Amazon S3 Bucket</em> for the input training and testing image and manifest files, as well as for the output files. We then proceed to transfer all the relevant files to their corresponding channel in the <em> S3 Bucket </em> by supplying the appropriate path. </p>

In [ ]:
# Unzipping the garbage_classification dataset 
!unzip -qq garbage_classification.zip

# Running the Python script to create the training and testing manifest files in their respective output folders
!python im2rec.py --list --recursive garbage_classification/garbage_train garbage_classification/garbage_train
!python im2rec.py --list --recursive garbage_classification/garbage_test garbage_classification/garbage_test

In [ ]:
# Creating variables for the location paths of the input training and testing image and .lst files accordingly, as well as the output files
train_path = 's3://{}/{}/input/training/'.format(bucket, prefix) 
test_path = 's3://{}/{}/input/testing/'.format(bucket, prefix) 

train_lst_path = 's3://{}/{}/input/training_lst/'.format(bucket, prefix) 
test_lst_path = 's3://{}/{}/input/testing_lst/'.format(bucket, prefix) 

out_path = 's3://{}/{}/output/'.format(bucket, prefix)

In [ ]:
# Uploading the image and manifest files to the training and testing channels accordingly
!aws s3 cp garbage_classification/garbage_train $train_path --recursive --quiet
!aws s3 cp garbage_classification/garbage_test $test_path --recursive --quiet

!aws s3 cp garbage_classification/garbage_train.lst $train_lst_path --quiet
!aws s3 cp garbage_classification/garbage_test.lst $test_lst_path --quiet

<h2>Build and Train the Model</h2>
<p style="text-align: justify;"> We are now ready to use <em>Amazon Sagemakr's Image Classification </em>algorithm. For this business problem, we will be using the supervised learning algorithm to solve the outlined label classification problem. This algorithm employs a convolutional neural network that takes an image as input and outputs a label assigned to the inputted image. More specifically, in terms of the given domain, the algorithm will output one of the six predefined classes, along with a probability for the outputted class. We configure the <em>Estimator</em> for the <em>Image Classification </em>algorithm and specify the <em>num_classes</em> hyperparameter to six and the <em>num_training_samples</em> hyperparameter to 2,020 since the dataset has six distinct classes and 2,020 training images. Note that these two hyperparameters are required and we explore the optional hyperparameters below. Afterwards, we define the training and testing data channels for the image and manifest files and set their <em>content_type</em> to "<em>application/x-image</em>". </p>

In [ ]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator 

role = get_execution_role()
container = retrieve('image-classification', region) 

# Initializing and configure the Amazon Sagemaker Image Classification Algorithm
ic_estimator = Estimator(
    container, 
    role=role, 
    instance_count=1, 
    instance_type='ml.p2.xlarge', 
    output_path=out_path)

<p style="text-align: justify;"> We now tune the model by supplying a <em>learning_rate</em> of 0.05 and a <em>mini_batch_size</em> of 25. The learning rate hyperparameter controls the speed at which the model learns, specifically, in response to the estimated error at the end of each batch of training images. Additionally, we set the number of <em>epochs</em> to 10 and the <em>checkpoint_frequency</em> to two, which tells the model to save a checkpoint file every two epochs, for a total of five checkpoints. We also <em>resize</em> the training images to 256 pixels wide and set the value of five for the <em>top_k</em> hyperparameter, which will report the top-five accuracy during training. We then specify the <em>num_layers </em>for the network to have 50 layers and set the flag to <em>use_pretrained_model</em> to 1 so that the network does not need to be trained from scratch. The model training process is further equipped with <em>early_stopping</em> logic so as to not overtrain the model on the training dataset. Finally, we experiment with the <em>augmentation_type</em> parameter in an attempt to build a more robust algorithm by randomly performing a series of crops, flips, transformations, and recolouring of the training images. The <em>Amazon Web Services </em>documentation states that this hyperparameter has the following valid values: <em>crop</em>, <em>crop_color</em>, and <em>crop_color_transform</em>. </p>

In [ ]:
# Setting the hyperparameters for the Amazon Sagemaker Image Classification Algorithm
ic_estimator.set_hyperparameters(
    num_classes          = 6,     # REQUIRED
    num_training_samples = 2020,  # REQUIRED
    augmentation_type    = 'crop_color_transform', 
    checkpoint_frequency = 2,
    early_stopping       = True, 
    epochs               = 10,
    learning_rate        = 0.05,
    mini_batch_size      = 25,
    num_layers           = 50, 
    top_k                = 5, 
    resize               = 256, 
    use_pretrained_model = 1) 

In [ ]:
from sagemaker import TrainingInput 

# Defining the training and testing data channels for the image and manifest files accordingly
train_data_channel = TrainingInput(s3_data=train_path, content_type='application/x-image')
test_data_channel = TrainingInput(s3_data=test_path, content_type='application/x-image') 

train_lst_data_channel = TrainingInput(s3_data=train_lst_path, content_type='application/x-image') 
test_lst_data_channel = TrainingInput(s3_data=test_lst_path, content_type='application/x-image')

channels = {
    'train': train_data_channel,
    'test': test_data_channel,
    'train_lst': train_lst_data_channel, 
    'test_lst': test_lst_data_channel}

<p style="text-align: justify;"> Here, we launch the training job by supplying all four data channels to the <em>fit()</em> API. Please note that due to the restrictions for an <em>Amazon Web Services</em> user with a <em>Lab Role</em>, we are unfortunately unable to train the model. To elaborate, the error stems from specifying the <em>instance_type</em> of the <em>Image Classification Estimator</em> to <em>'ml.p2.xlarge</em>', which exceeds the resource limits of this account. Otherwise, a series of messages would be outputted to the console that log the model training process and conclude by notifying that the training job is completed, along with the length of time required to train the model, in seconds.  </p>

In [ ]:
# Launching the training job by supplying all four data channels to the fit() API
ic_estimator.fit(inputs=channels)

# ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateTrainingJob operation: 
# The account-level service limit 'ml.p2.xlarge for training job usage' is 0 Instances, 
# with current utilization of 0 Instances and a request delta of 1 Instances. 
# Please contact AWS support to request an increase for this limit.

<h2>Deploy the Model</h2>
<p style="text-align: justify;"> We are ready to deploy our model in <em>Amazon Sagemaker</em>. We configure and launch a new training job and set the <em>instance_type</em> to '<em>ml.t2.medium</em>', a smaller instance to save money. Remember to always delete the endpoint after testing the model to avoid unnecessary costs. Again, please note that we are unable to deploy the model as a direct consequence of not being able to train the model. The resulting error notifies that the <em>Image Classification Estimator</em> is not associated with a training job. </p>

In [ ]:
# Deploying the model
ic_predictor = ic_estimator.deploy(
    initial_instance_count=1, 
    instance_type='ml.t2.medium')

# ValueError: Estimator is not associated with a training job

<h2>Demo the Model</h2>
<p style="text-align: justify;"> Below we supply the model with a new input image. Note that a folder, containing seven test images, has been included in this environment, named '<em>input_images</em>'. It is important to change the file name of the image, '<em>input_images/input_image#.jpg</em>', to one of the corresponding input image file names. Alternatively, the code may be modified to accept an entire folder in which the contents of the folder will be recursively iterated over.</p>

<p style="text-align: justify;"> Although we are unable to demonstrate the model in this notebook, we know that the model will output a probability followed by a class label. The probability can be interpreted as the level of confidence for the model's classification of a given image, ranging from 0 to 100 percent. The table lookup below illustrates the class of waste and its corresponding label number. Recall that these labels were generated via the manifest files earlier.</p>

<table border="1" cellspacing="0" cellpadding="0">
<tbody>
<tr>
<td valign="top" width="91">
<p align="center"><strong>Class</strong></p>
</td>
<td valign="top" width="129">
<p align="center"><strong>Label</strong></p>
</td>
</tr>
<tr>
<td valign="top" width="91">
<p align="right">Cardboard</p>
</td>
<td valign="top" width="129">
<p align="center">0</p>
</td>
</tr>
<tr>
<td valign="top" width="91">
<p align="right">Glass</p>
</td>
<td valign="top" width="129">
<p align="center">1</p>
</td>
</tr>
<tr>
<td valign="top" width="91">
<p align="right">Metal</p>
</td>
<td valign="top" width="129">
<p align="center">2</p>
</td>
</tr>
<tr>
<td valign="top" width="91">
<p align="right">Paper</p>
</td>
<td valign="top" width="129">
<p align="center">3</p>
</td>
</tr>
<tr>
<td valign="top" width="91">
<p align="right">Plastic</p>
</td>
<td valign="top" width="129">
<p align="center">4</p>
</td>
</tr>
<tr>
<td valign="top" width="91">
<p align="right">Trash</p>
</td>
<td valign="top" width="129">
<p align="center">5</p>
</td>
</tr>
</tbody>
</table>

In [ ]:
!unzip -qq input_images.zip

<p style="text-align: justify;"> Here are the seven sample input images:</p>

<img align="left" src="input_images/input_image1.jpg" style="width:150px;height:150px;">
<img align="left" src="input_images/input_image2.jpg" style="width:150px;height:150px;">
<img align="left" src="input_images/input_image3.jpg" style="width:150px;height:150px;">
<img align="left" src="input_images/input_image4.jpg" style="width:150px;height:150px;">
<img align="left" src="input_images/input_image5.jpg" style="width:150px;height:150px;">
<img align="left" src="input_images/input_image6.jpg" style="width:150px;height:150px;">
<img align="left" src="input_images/input_image7.jpg" style="width:150px;height:150px;">

In [ ]:
import json 
import numpy as np 

# Testing the image classification against a set of new images, defined below
# REQUIRED: Change 'input_images/input_image#.jpg' to the .jpg file of the input image
with open('input_images/input_image#.jpg', 'rb') as f: 
    payload = f.read() 
    payload = bytearray(payload) 
runtime = boto3.Session().client( 
    service_name='runtime.sagemaker') 
response = runtime.invoke_endpoint( 
    EndpointName=ic_predictor.endpoint_name,
    ContentType='application/x-image', 
    Body=payload) 
result = response['Body'].read() 
result = json.loads(result) 
index = np.argmax(result) 
print(result[index], index)

# NameError: name 'ic_predictor' is not defined

In [ ]:
# Tearing down the endpoint to avoid unnecessary costs
ic_predictor.delete_endpoint(EndpointName=endpoint_name)

# NameError: name 'ic_predictor' is not defined

<p style="text-align: justify;"> Finally, returning to the original implementation of the waste classification system in AccuWaste's sorting facilities, the level of confidence outputted by the model should be used as a preliminary tool for decision-making. For example, AccuWaste may choose to only accept classifications with greater than 80% accuracy and reject all other classifications. Note that since the company's objective was to reduce the amount of recyclable material being treated as trash, thus, it is imperative for the management team at AccuWaste to understand the costs associated with each type of misclassification error. On one hand, a false positive (type I error) is when the system classifies an item as trash when it is in fact a recyclable material. On the other hand, a false negative (type II error) is when the system classifies an item as a recyclable material when it is in fact trash. From this, we see that type I errors may be preferred over type II errors since the cost of a type I error is an increase in the cost of disposal (the metric under observation) whereas the cost of a type II error may lead to contamination of actual recyclable material at the recycling center and create an array of inconsistencies.</p>

<p style="text-align: justify;"> Therefore, AccuWaste must factor in the risk it is willing to take when classifying images. The company must identify its degree of risk tolerance and interpret the model output accordingly. Regardless of this tolerance, the company should continue to gather images of the types of materials that the sorting facility frequently encounters. By providing a larger dataset with more training images, the model's accuracy will improve as a result and, thus, AccuWaste can shift the focus of their efforts to other areas of concern knowing that their sorting facility is capable of accurately classifying and sorting waste, with minimal human interaction. </p>

<h2>Sources</h2>
<p style="text-align: justify;"> In this section include any sources of inspiration, how it inspired you and the URL. </p>

<p style="text-align: justify;"> The following Amazon Web Services documentation for Amazon Sagemaker's Image Classification Algorithm was useful when initializing, configuring, and troubleshooting the Amazon Sagemaker environment. </p>
<p><a href="https://docs.aws.amazon.com/sagemaker/latest/dg/image-classification.html"> https://docs.aws.amazon.com/sagemaker/latest/dg/image-classification.html </a></p>

</br>

<p style="text-align: justify;"> The following Amazon Web Services documentation for Image Classification Hyperparameters was useful to better understand the various hyperparamaters. </p>
<p><a href="https://docs.aws.amazon.com/sagemaker/latest/dg/IC-Hyperparameter.html"> https://docs.aws.amazon.com/sagemaker/latest/dg/IC-Hyperparameter.html </a></p>

</br>

<p style="text-align: justify;"> Several of the code blocks relating to setting up the environment, configuring and launching the training job as well as deploying and testing the model were taken from the <em>HotDog.ipynb</em> file provided in class. Similarly, the supplemental workshop that explored the <em>Eyes.ipynb</em> file was referenced, especially for troubleshooting purposes. </p>
